In [57]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [58]:
# Set csv paths
trial_path = 'raw_data/clinicaltrial_data.csv'
drug_path = 'raw_data/mouse_drug_data.csv'

In [59]:
# Read trial csv
trial_df = pd.read_csv(trial_path)
trial_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [60]:
trial_df.count()

Mouse ID              1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
dtype: int64

In [61]:
# Check duplicate mouse
mouse_id_index = trial_df.set_index(['Mouse ID'])
mouse_check = mouse_id_index.loc['g989',:]
mouse_check

,Timepoint,Tumor Volume (mm3),Metastatic Sites
Mouse ID,,,
g989,0,45.000000,0
g989,0,45.000000,0
g989,5,48.786801,0
g989,5,47.570392,0
g989,10,51.745156,0
g989,10,49.880528,0
g989,15,51.325852,1
g989,15,53.442020,0
g989,20,55.326122,1


In [62]:
# Will decide to assign a new id to duplicate, check indices
trial_df.index[trial_df['Mouse ID'] == 'g989'].tolist()

[107, 137, 329, 360, 620, 681, 815, 869, 950, 1111, 1195, 1380, 1592]

In [75]:
# Assign new mouse id to duplicate
trial_df.set_value([137, 360, 681, 869, 1111], 'Mouse ID', 'g989-1')

# Not sure which drug was administered to which mouse based on these values, need to check data
# 107, 329, 620, 815, 950, 1195, 1380, 1592

# Check new mouse id at indices
trial_df['Mouse ID'][[137, 360, 681, 869, 1111]]

137     g989-1
360     g989-1
681     g989-1
869     g989-1
1111    g989-1
Name: Mouse ID, dtype: object

In [64]:
# Read drug csv
drug_df = pd.read_csv(drug_path)
drug_df.head()

,Mouse ID,Drug
0,f234,Stelasyn
1,x402,Stelasyn
2,a492,Stelasyn
3,w540,Stelasyn
4,v764,Stelasyn


In [65]:
drug_df.count()

Mouse ID    250
Drug        250
dtype: int64

In [66]:
# Check duplicate mouse
drug_df[drug_df['Mouse ID'].duplicated(keep=False)]

,Mouse ID,Drug
7,g989,Stelasyn
173,g989,Propriva


In [67]:
# Will decide to assign a new id to duplicate
drug_df.set_value(173, 'Mouse ID', 'g989-1')

# Check changed mouse
drug_df['Mouse ID'][173]

'g989-1'

In [76]:
# Try merging the two dfs
merged_trial_drug = pd.merge(trial_df, drug_df, on='Mouse ID')
merged_trial_drug.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug
0,b128,0,45.000000,0,Capomulin
1,b128,5,45.651331,0,Capomulin
2,b128,10,43.270852,0,Capomulin
3,b128,15,43.784893,0,Capomulin
4,b128,20,42.731552,0,Capomulin


In [69]:
# Check count
merged_trial_drug.count()

Mouse ID              1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
Drug                  1893
dtype: int64

In [77]:
# Check duplicate mouse
mouse_id_index = merged_trial_drug.set_index(['Mouse ID'])
mouse_check = mouse_id_index.loc['g989-1',:]
mouse_check

,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug
Mouse ID,,,,
g989-1,0,45.000000,0,Propriva
g989-1,5,47.570392,0,Propriva
g989-1,10,49.880528,0,Propriva
g989-1,15,53.442020,0,Propriva
g989-1,20,54.657650,1,Propriva
